In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
from rts.db_settings import DATABASE_URL
from rts.db.dao import DataAccessObject
from rts.pipeline import PipelineIOC

DataAccessObject().connect(DATABASE_URL)


In [4]:
metadata = '../data/metadata.hdf5'
with pd.HDFStore(metadata) as store:
    keys = store.keys()
    df = store.get(keys[0])  

In [5]:
# This preprocessing step should be outside of the pipeline code and come with the dataframe
def find_mp4_files(root_folder):
    import os
    mp4_files = []

    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.mp4'):
                mp4_files.append(os.path.join(foldername, filename))

    return mp4_files

mp4_files = find_mp4_files("/media/data/ioc/videos/")
file_map = {}
for mp4 in mp4_files:
    file_map[mp4.split('/')[-1].split('.')[0]] = mp4

df['path'] = df['guid'].apply(lambda x: file_map[x] if x in file_map else None)

In [ ]:
# Process a single video and upload clips. This funcction would also take a dataframe that has more than one video
PipelineIOC().ingest(df[df.guid == 'MDA42810489'], notebook=True)